#Ingestion del archivo language.csv


### Paso 1 - Leer el archivo CSV usando "DataframeReader"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType

In [0]:
dbutils.widgets.text("p_environment", "")
v_enviroment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run 
"../includes/common_functions"



In [0]:
%run
"../includes/configuration"

In [0]:
language_schema = StructType(fields=[
    StructField("languageId", IntegerType(), False),
    StructField("languageCode", StringType(), True),
    StructField("languageName", StringType(), True),
    ])

In [0]:
language_df = spark.read \
    .option("header", True)\
    .schema(language_schema)\
    .csv(f"{bronze_folder_path}/{v_file_date}/language.csv")

In [0]:
display(language_df)


In [0]:
display(language_df.describe())

## Parte 2 : Seleccionar solo las columnas requeridas

In [0]:
language_selected_df = language_df.select("languageId", "languageName")

In [0]:
from pyspark.sql.functions import col


## Paso 3. Cambiar el nombre de las columnas segun lo requerido

In [0]:
language_rename_df = language_selected_df \
    .withColumnRenamed("languageId", "language_id") \
    .withColumnRenamed("languageName", "language_name")
display(language_rename_df)

    

### Paso 4. Agregar la columna "Ingestion date" al dataframe

In [0]:
from pyspark.sql.functions import current_timestamp, lit
# se importan funciones de fecha del dia de ingestion en el sistema y lit que nos sirve para agragar un valor en una columna ya que si se hace sin esta funcion no sera tomada como objeto



In [0]:
language_final_df = language_rename_df \
    .withColumn("ingestion_date", current_timestamp()) \
    .withColumn("enviroment", lit(v_enviroment)) \
    .withColumn("file_date", lit(v_file_date))


### Paso 5. Escribir datos en el datalake en formato "Parquet"

In [0]:
language_final_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.languages")

In [0]:
%fs
ls /mnt/moviehistory7809/silver/languages

In [0]:
%sql
select * from movie_silver.languages


In [0]:
dbutils.notebook.exit("Ejecutado exitosamente")